In [ ]:

from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Dropout, merge, BatchNormalization
import numpy as np
import keras.backend as K
from keras.models import load_model
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.layers import concatenate,Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
#K.set_image_dim_ordering('tf')
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Dropout, merge, MaxPool2D
import numpy as np
import keras.backend as K
from keras.models import load_model
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import to_categorical
from keras.layers import concatenate,Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
#K.set_image_dim_ordering('tf')
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.backend.tensorflow_backend import set_session
K.set_image_data_format(data_format='channels_first')
'''config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))'''
#K.set_image_dim_ordering('tf')

'''train = np.load('/home/ubuntu/MobileExtraction/Train256x324-full.npy')
labels = np.load('/home/ubuntu/MobileExtraction/Labels256x324-full.npy')
print(np.max(train),np.min(train))
x = np.argwhere(labels>0)
print(x.shape)
crf_labels = labels.flatten()
for i in range(x.shape[0]):
    labels[x[i][0],x[i][1],x[i][2]] = 1
train = train.reshape(train.shape[0],256,324)
trainNew = np.empty((394,256,324,3),dtype='float32')
trainNew[:,:,:,0]=train
trainNew[:,:,:,1]=train
trainNew[:,:,:,2]=train
print(np.unique(labels))
import tensorflow as tf
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)
print(labels.shape)
print(trainNew.shape)
labels=labels.astype('uint8')'''
smooth = 1.

#labels = to_categorical(y=labels,n)
#import tensorflow as tf
smooth = 1.
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)-K.log(dice_coef(y_true, y_pred))


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return 1-jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

import h5py
from keras.models import Model
from keras.layers import Input, Activation, Concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from skimage import io
import cv2
import numpy as np

def pad_input(input_img,patch_size=[256,256]):
    h = input_img.shape[0]
    w = input_img.shape[1]
    hMod = h % patch_size[0]
    wMod = w % patch_size[1]
    hPad=0
    wPad=0
    if(hMod!=0 or wMod!=0):
        hPad = patch_size[0]-hMod
        wPad = patch_size[1]-wMod
    output = np.pad(input_img, ((0, hPad),(0,wPad),(0,0)), 'constant')
    return(output)

def prediction_patches(img,patch_size=[256,256]):
    img = np.asarray(img).astype('float32')
    #print(img.shape)
    assert(img.shape[0]%patch_size[0]==0 and img.shape[1]%patch_size[1]==0)
    h_patch = int(img.shape[0]/patch_size[0])
    w_patch = int(img.shape[1]/patch_size[1])
    patches = []
    x = 0
   
    no = 0
    for i in range(h_patch):
        y = 0 
        for j in range(w_patch):
            
            
            patch = img[x:(((i+1)*patch_size[0])),y:(((j+1)*patch_size[1])),:]
            #print(y)
            #print(patch.shape)
            #print(j)
            y = ((j+1)*patch_size[1])
            #print('yes')
            no+=1
            patches.append(patch)
        x = ((i+1)*patch_size[0])
    
    return patches

def reconstruct_patches(patches= np.ones((16,256,256,3)),img_shape=[1024,1024,3]):
    full_img = np.empty(img_shape).astype('float32')
    h = img_shape[0]
    w = img_shape[1]
    c = img_shape[2]
    h_patch = int(h / patches.shape[1])
    w_patch = int(w / patches.shape[2])
    no = 0 
    x = 0
    
    #assert(h_patch*w_patch == patches.shape[0])
    for i in range(h_patch):
        y = 0
        for j in range(w_patch):
            full_img[x:(((i+1)*patches.shape[1])),y:((j+1)*patches.shape[2]),:] = patches[no]
            #print('yes')
            y = ((j+1)*patches.shape[2])
            no += 1  
        x = ((i+1)*patches.shape[1])
    return full_img

def LargeDilatedKernels(input_shape):
    inputs = Input((input_shape))
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(inputs)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(64,(3,3),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(256,(7,7),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(256,(1,1),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-5))(x)
    x = BatchNormalization(axis=1)(x)
    x = Conv2D(1,(1,1),activation='sigmoid',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-5))(x)
    model = Model(inputs=[inputs],outputs=[x])
    return model

def dilatedKernels(input_shape):
    inputs = Input((input_shape))
    conv1 = Conv2D(64,(7,7),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(inputs)
    conv2 = Conv2D(64,(7,7),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv1)
    conv3 = Conv2D(64,(7,7),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv2)
    conv4 = Conv2D(64,(7,7),activation='relu',dilation_rate=(4,4),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv3)
    concat1 = concatenate([conv1,conv2,conv3,conv4],axis=1 )
    drop1 = Dropout(0.5)(concat1)
    conv11 = Conv2D(64,(5,5),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(drop1)
    conv12 = Conv2D(64,(5,5),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv11)
    conv12 = Dropout(0.5)(conv12)
    conv13 = Conv2D(64,(5,5),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv12)
    conv14 = Conv2D(64,(5,5),activation='relu',dilation_rate=(4,4),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv13)
    concat2 = concatenate([conv11,conv12,conv13,conv14],axis=1 )
    drop2 = Dropout(0.5)(concat2)
    conv21 = Conv2D(64,(3,3),activation='relu',padding='same',dilation_rate=(1,1),kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(drop2)
    conv22 = Conv2D(64,(3,3),activation='relu',padding='same',dilation_rate=(2,2),kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv21)
    conv22 = Dropout(0.5)(conv22)
    conv23 = Conv2D(64,(3,3),activation='relu',padding='same',dilation_rate=(3,3),kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv22)
    conv24 = Conv2D(64,(3,3),activation='relu',padding='same',dilation_rate=(4,4),kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv23)
    concat3 = concatenate([conv21,conv22,conv23,conv24],axis=1 )
    drop3 = Dropout(0.5)(concat3)
    conv31 = Conv2D(64,(1,1),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(drop3)
    conv32 = Conv2D(64,(1,1),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv31)
    conv32 = Dropout(0.5)(conv32)
    conv33 = Conv2D(64,(1,1),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv32)
    conv34 = Conv2D(64,(1,1),activation='relu',dilation_rate=(4,4),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv33)
    concat4 = concatenate([conv31,conv32,conv33,conv34],axis=1 )
    drop4 = Dropout(0.5)(concat4)
    conv41 = Conv2D(64,(3,3),activation='relu',dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(drop4)
    conv42 = Conv2D(64,(3,3),activation='relu',dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv41)
    conv42 = Dropout(0.5)(conv42)
    conv43 = Conv2D(64,(3,3),activation='relu',dilation_rate=(3,3),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv42)
    conv44 = Conv2D(64,(3,3),activation='relu',dilation_rate=(4,4),padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-5))(conv43)
    concat5 = concatenate([conv44,conv34,conv24,conv14,conv4],axis=1 )
    drop5 = Dropout(0.5)(concat5)
    final1 = Conv2D(128,(3,3),dilation_rate=(3,3),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-5))(drop5)
    final1 = Conv2D(128,(1,1),dilation_rate=(1,1),activation='relu',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-5))(final1)
    final1 = Dropout(0.5)(final1)
    finallast = Conv2D(1,(1,1),dilation_rate=(1,1),activation='sigmoid',padding='same',kernel_initializer='glorot_uniform',kernel_regularizer=l2(1e-5))(final1)
    model = Model(inputs=[inputs],outputs=[finallast])
    return model
def encoder_decoder(input_shape):
    return Sequential([
    
        Conv2D(32, (3, 3),dilation_rate=(3,3), activation='relu', padding='same',input_shape=input_shape),
        
        MaxPooling2D((2,2), strides=(2,2)),
        Dropout(0.3),
        Conv2D(64, (3, 3),dilation_rate=(2,2), activation='relu' ,padding='same',kernel_initializer='he_normal'),
      
        MaxPooling2D((2,2), strides=(2,2)),
        Dropout(0.3),
        Conv2D(128, (3, 3),dilation_rate=(1,1), activation='relu',padding='same',kernel_initializer='he_normal'),
        MaxPooling2D((2,2), strides=(2,2)),
        Dropout(0.3),
        Conv2D(256, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'),
        MaxPooling2D((2,2), strides=(2,2)),
        Dropout(0.3),
        Conv2D(512, (3, 3), activation='relu', padding='same',kernel_initializer='he_normal'),
      
        UpSampling2D(size=(2,2)),
        Dropout(0.3),
        Conv2D(256, (3, 3),dilation_rate=(1,1), activation='relu', padding='same',kernel_initializer='he_normal'),
      
        UpSampling2D(size=(2,2)),
        Dropout(0.3),
        Conv2D(128, (3, 3),dilation_rate=(2,2),activation='relu', padding='same',kernel_initializer='he_normal'),
       
        UpSampling2D(size=(2,2)),
        Dropout(0.3),
        Conv2D(64, (3, 3),dilation_rate=(3,3), activation='relu',padding='same',kernel_initializer='he_normal'),
        UpSampling2D(size=(2,2)),
        Dropout(0.3),
        Conv2D(32, (3, 3),dilation_rate=(3,3), activation='relu',padding='same',kernel_initializer='he_normal'),
        Conv2D(1, (1, 1), activation='sigmoid')
    ])
inputs = Input((3,256,256))
x = Conv2D(filters=32,activation='relu',dilation_rate=(1,1),kernel_size=(7,7),padding='same',kernel_regularizer=l2(1e-5))(inputs)
x = BatchNormalization()(x)
x1 = Conv2D(filters=32,activation='relu',dilation_rate=(2,2),kernel_size=(5,5),padding='same',kernel_regularizer=l2(1e-5))(x)
x1 = BatchNormalization()(x1)
#x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(filters=32,activation='relu',dilation_rate=(3,3),kernel_size=(3,3),padding='same',kernel_regularizer=l2(1e-5))(x1)
x = BatchNormalization()(x)

#x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',kernel_initializer='he_normal')(x)
#concat = concatenate([x1,x2,x3,x4],axis=1 )
#x = Dropout(0.5)(x)
x = Conv2D(filters=256,activation='relu',kernel_size=(3,3),dilation_rate=(3,3),padding='same',kernel_regularizer=l2(1e-5))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Conv2D(filters=1,activation='sigmoid',kernel_size=(1,1),dilation_rate=(1,1),padding='same',kernel_regularizer=l2(1e-5))(x)
modelDilated = Model(inputs=inputs,outputs=x)

def get_unet():
    inputs = Input((3,256, 256))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.5)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPool2D(pool_size=(2,2))(conv1)
    pool1 = BatchNormalization()(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.3)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPool2D(pool_size=(2,2))(conv2)
    pool2 = BatchNormalization()(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.5)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPool2D(pool_size=(2,2))(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPool2D(pool_size=(2,2))(conv4)
    pool4 = BatchNormalization()(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Dropout(0.5)(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same',activation='relu',kernel_regularizer=l2(1e-6))(conv5), conv4], axis=1 )
    up6 = BatchNormalization()(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Dropout(0.5)(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same',activation='relu',kernel_regularizer=l2(1e-6))(conv6), conv3], axis=1 )
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Dropout(0.5)(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same',activation='relu',kernel_regularizer=l2(1e-6))(conv7), conv2], axis=1 )
    up8 = BatchNormalization()(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Dropout(0.5)(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same',kernel_regularizer=l2(1e-6), kernel_initializer='he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same',activation='relu',kernel_regularizer=l2(1e-6))(conv8), conv1], axis=1 )
    up9 = BatchNormalization()(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-6))(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Dropout(0.5)(conv9)
    
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',kernel_regularizer=l2(1e-6))(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = Conv2D(1, (1, 1), activation='sigmoid',kernel_regularizer=l2(1e-6))(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5,decay=5e-6), loss=bce_dice, metrics=[dice_coef])

    return model


In [ ]:
#modelDilated = dilatedKernels(input_shape=(3,256,256))
print(modelDilated.summary())
#history = model.fit(train,labels,batch_size=1,epochs=100,shuffle=True,validation_split=0.3)

In [ ]:

fcl_loss = focal_loss()
modelDilated.compile(optimizer=Adam(lr=1e-5),loss=bce_dice,metrics=[jacard_coef, dice_coef])

In [ ]:
model1 = encoder_decoder(input_shape=(1024,1024,3))

In [ ]:
#model1.compile(optimizer=Adam(lr=1e-3,decay=5e-4), loss=bce_dice, metrics=[dice_coef,jacard_coef])
checkpoint = ModelCheckpoint(filepath='/home/ubuntu/DeepMobile/models/densenet/LargeDilatedKernels-scratch.h5',mode='max',
                             monitor='val_dice_coef',verbose=1,save_best_only=True)
#modelDilated.load_weights('/home/ubuntu/DeepMobile/models/unet/unetNoMaxPool3.h5')
# we create two instances with the same arguments-

# we create two instances with the same arguments
data_gen_args = dict(rotation_range=90.,horizontal_flip=True,vertical_flip=True,
                     zoom_range=0.2,fill_mode='mirror')
image_datagen = ImageDataGenerator(rescale=1/255.,**data_gen_args)
mask_datagen = ImageDataGenerator(rescale=1/255.,**data_gen_args)
mask_datagen1 = ImageDataGenerator(rescale=1/255.,**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 30

image_generator = image_datagen.flow_from_directory(
    '/home/ubuntu/DeepMobile/data/scratches/train/',target_size=(256, 256),
    class_mode=None, batch_size=1,color_mode='rgb',
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    '/home/ubuntu/DeepMobile/data/scratches/label/',
    class_mode=None,batch_size=1,target_size=(256, 256),color_mode='grayscale',
    seed=seed)
image_generator1 = image_datagen.flow_from_directory(
    '/home/ubuntu/DeepMobile/data/scratches/test_Data/train/',target_size=(256, 256),
    class_mode=None, batch_size=1,color_mode='rgb',
    seed=seed)

mask_generator1 = mask_datagen.flow_from_directory(
    '/home/ubuntu/DeepMobile/data/scratches/test_Data/labels/',
    class_mode=None,batch_size=1,target_size=(256, 256),color_mode='grayscale',
    seed=seed )

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)
valid_generator = zip(image_generator1, mask_generator1)
modelDilated.fit_generator(
    train_generator,
    steps_per_epoch=10000,validation_data=valid_generator,validation_steps=100,
    epochs=100,callbacks=[checkpoint])

In [ ]:
data_gen_args = dict(height_shift_range=0.3,width_shift_range=0.3,fill_mode='mirror',horizontal_flip=True,vertical_flip=True,
                     rotation_range=90.,
                     )
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1
image_datagen.fit(trainNew, augment=True, seed=seed)


# combine generators into one which yields image and masks
train_generator = zip(image_datagen.flow(x=trainNew[:200],batch_size=1,seed=1), mask_datagen.flow(x=labels[:200],batch_size=1,seed=1))
valid_generator = zip(image_datagen.flow(x=trainNew[200:],batch_size=1,seed=1), mask_datagen.flow(x=labels[200:],batch_size=1,seed=1))

model.fit_generator(callbacks=[checkpoint],validation_data=valid_generator,validation_steps=50,
    generator=train_generator,
    steps_per_epoch=200,
    epochs=100)

In [ ]:
from skimage import io as skio
import matplotlib.pyplot as io
from glob import glob
import cv2
testPath = glob('/home/ubuntu/DeepMobile/data/scratches/blacknwhite/2/353513070724013_back.png')
testPath.sort()
h=256
w=256
no = 182
'''def FillImg(binImg):
    _, contours, hieqrarchy = cv2.findContours(np.uint8(binImg), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    newBinImg = np.zeros((binImg.shape[0],binImg.shape[1]),np.uint8())
    areas = [cv2.contourArea(c) for c in contours]
    if(len(areas)>0):
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        cv2.drawContours(np.uint8(newBinImg), [cnt], 0, 255, -1)
        
    return newBinImg
'''
for i in range(len(testPath)):
    #tokens = testPath[i].split('/)
    #name = tokens[6].split('.')[0]
    img =cv2.imread(testPath[i],3)

    shape = img.shape
    img = np.asarray(img).astype('uint8')
   
    
    print(img.shape)
    #img = cv2.resize(img,dsize=(w,h),interpolation=cv2.INTER_AREA)
    #img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    #print(img.shape)
    #label = cv.resize(label,dsize=(324,256))

    #label = label.astype('uint8')
    #img = img.reshape(1,h,w,3)

    #print(img.shape)
    #imgss = img
    
    img = img/255.
    patches = prediction_patches(pad_input(img,patch_size=[h,w]),patch_size=[h,w])
    patches = np.asarray(patches)
    #label = label.reshape(512,648,1)
    print(patches.shape)
    patches = patches.transpose([0,3,1,2])
    
    skio.imshow(img)
    skio.show()
    #img = (img-np.mean(img))/np.std(img)
    padded_input = pad_input(img)
    shape = padded_input.shape
    print(patches.shape)
    predicted_patches = modelDilated.predict(patches,batch_size=1,verbose=1)
    predicted_patches = predicted_patches.transpose([0,2,3,1])
    predicted = reconstruct_patches(img_shape=shape,patches=predicted_patches)

    print(np.max(predicted),np.min(predicted))
    print(predicted.shape)
    io.imshow(predicted)
    io.show()
    thresh = np.argwhere(predicted>0.2)
    imgs = np.zeros(shape).astype('uint8')
    print(thresh.shape)
    for i in range(thresh.shape[0]):
        imgs[thresh[i][0],thresh[i][1],0]=255
    
    skio.imshow(imgs[:,:,0])
    skio.show()
    '''io.imsave('/home/ubuntu/DeepMobile/data/output_scratches/original/dilated__{}.png'.format(int(no)),img)
    io.imsave('/home/ubuntu/DeepMobile/data/output_scratches/predicted/dilated__{}.png'.format(int(no)),imgs[:,:,0])
    no+=1'''

In [ ]:
img  = img/255.
img = (img- np.mean(img))/ np.std(img)
img = img.reshape(1,320,480,3)

In [ ]:
out = model.predict(img)
print out.shape
out = out.transpose([0,3,1,2])
io.imshow(out[0][0])

In [ ]:
np.unique(labels)

In [ ]:
from skimage import io as skio
import matplotlib.pyplot as io
from glob import glob
testPath = glob('/home/ubuntu/De')
for i in xrange(len(testPath)):
    tokens = testPath[i].split('/')
    name = tokens[6].split('.')[0]
    img = skio.imread(testPath[i])
    io.imshow(img)
    io.show()
    img  = img/255.
    img = (img-np.mean(img))/np.std(img)
    if img.shape==(320,480,3):
        print 'yes'
    else:
        img = img.transpose([1,0,2])
    img = img.reshape(1,320,480,3)
    x = np.random.normal(loc=0.0, scale=1.0, size=(256,256,3))
    predicted = model.predict(img)
    predicted = predicted.transpose([0,3,1,2])
    io.imshow(predicted[0][0])
    io.show()
    thresh = np.argwhere(predicted>0.000001)
    for i in xrange(thresh.shape[0]):
        predicted[thresh[i][0],thresh[i][1],thresh[i][2],thresh[i][3]]=255
    
    io.imshow(predicted[0][0])
    io.show()
    

In [ ]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

In [ ]:
get_model_memory_usage(4,model)

In [ ]:
tiramisu

In [ ]:
#model.save('/home/ubuntu/DeepMobile/models/unet/unetNoMaxPool2.h5')
modelDilated.save_weights('/home/ubuntu/DeepMobile/models/unet/unetNoMaxPool3.h5')

In [ ]:
import cv2
cv2.namedWindow("preview")
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

while rval:
    
    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: # exit on ESC
        break
    #img = np.asarray(img).astype('float32')
    img = cv2.resize(frame,dsize=(1024,1024),interpolation=cv2.INTER_AREA)

    #label = cv.resize(label,dsize=(324,256))

    #label = label.astype('uint8')
    img = img.reshape(1,1024,1024,3)
    imgss = img
    img = img/255.
    #label = label.reshape(512,648,1)

    
    #img = (img-np.mean(img))/np.std(img)
    predicted = model.predict(img)
 
    
    thresh = np.argwhere(predicted[:,:,:,0]>0.9)
    imgs = np.zeros((1024,1024,3)).astype('uint8')
   
    for i in range(thresh.shape[0]):
        
        imgs[thresh[i][1],thresh[i][2],0]=imgss[0,thresh[i][1],thresh[i][2],0]
        imgs[thresh[i][1],thresh[i][2],1]=imgss[0,thresh[i][1],thresh[i][2],1]
        imgs[thresh[i][1],thresh[i][2],2]=imgss[0,thresh[i][1],thresh[i][2],2]
    cv2.imshow("preview", imgs)

cv2.destroyWindow("preview")
    

In [ ]:
model.save_weights('/home/ubuntu/DeepMobile/models/myown/unetOverfit.h5')

In [ ]:
x = np.random.normal(loc=0.0, scale=1.0, size=(256,256,3))

In [ ]:
x.shape


In [ ]:
x = io.imread('/home/ubuntu/DeepMobile/data/scratches/label/label/1000.png')

In [ ]:
np.max(x)

In [ ]:
np.min(x)